In [49]:
function compute_Tk(A,q,kmax)
    T = zeros(kmax, kmax)
    r = copy(q[:,1])
    beta = norm(r)
    q1 = zeros(n)

    for k = 1:kmax        
        q0 = copy(q1)
        q1 = r/beta
        r = A * q1
        alpha = dot(q1, r)
        T[k,k] = alpha
        if k > 1
            T[k-1,k] = beta
            T[k,k-1] = beta
        end
        r = r - alpha*q1 - beta*q0
        beta = norm(r)
    end
    
    return T
end

function eig_k(T,k)
    Tk = T[1:k,1:k]
    L, = eig(Tk) # Eigenvalue estimates at step k        
    return L    
end

eig_k (generic function with 1 method)

In [50]:
n = 128
srand(2016)

Q, = qr(rand(n,n))

D = cos(linspace(0,π,n))
D[div(n,2)] = 2
# D = cos(linspace(0,π/2,n))

x = linspace(0,1,n)
# D = x
# D = x.^2
# D = x.^2/2 - x.^3/3
# D = x.^5/5 - x.^4/2 + x.^3/3
# D = x.^9/9 - x.^8/2 + 6/7*x.^7 - 2/3*x.^6 + x.^5/5

# D = linspace(-1,1,n).^3

A = Q * diagm(D) * Q'

kmax = n # Size of Arnoldi space

q = zeros(n,kmax)
# Random starting vector
q[:,1] = rand(n)
q[:,1] /= norm(q[:,1])

Tk = compute_Tk(A,q,kmax);

In [51]:
# Testing accuracy
D, = eig(A); D1, = eig(Tk)
sort!(D); sort!(D1)
@show norm(D-D1) / norm(D)

norm(D - D1) / norm(D) = 0.24481064515651824


0.24481064515651824

In [52]:
using Plots
plotlyjs()

L = zeros(n,kmax)
for k=1:kmax
    x = eig_k(Tk,k)
    sort!(x)
    for l=1:k
        if l%2 == 1
            L[l,k] = x[(l+1)>>1]
        else
            L[l,k] = x[k-(l-2)>>1]
        end
    end
end

plot(1:kmax,L[1,:]',left_margin=30mm)
for k=2:kmax-1
    plot!(k:kmax,L[k,k:end]')
end
plot!([kmax],[L[kmax,kmax]])

scatter!(kmax*ones(n),D,markersize=2,lab="Exact")
xaxis!("Iteration")
yaxis!("Eigenvalue estimates")

In [53]:
generate_tex_pgfplot = false 
if generate_tex_pgfplot
    @eval import PGFPlots
    tr = [ PGFPlots.Plots.Scatter((1+kmax)*ones(n),D,legendentry="exact",markSize=1) ]
    for k=1:kmax
        push!(tr, PGFPlots.Plots.Linear(k:kmax,vec(L[k,k:end]),mark="none") )
    end

    ax = PGFPlots.Axis(tr, xlabel="Iteration", ylabel="Eigenvalue",
        legendPos="south west")    
end   

In [54]:
if generate_tex_pgfplot
    PGFPlots.save("Lanczos2.tex", ax, include_preamble=false)
end